# Error Propagation on Corrupted Cipher Text

1. Encrypt a <font color="yellow">64 bytes</font> long text file with <font color="yellow">AES-128</font>;

2. Flip a single bit, let’s say in the <font color="yellow">28th</font> byte, corrupted, which means you need to modify one bit in the cipher text;

3. Decrypt the modified cipher text using the correct key and IV;

4. How much info can you recover by decrypting the corrupted file, if the encryption mode is ECB, CBC, CFB, OFB, and CTR?

In [1]:
# visualization tools
import textwrap

color2num = dict(
    gray=30,
    red=31,
    green=32,
    yellow=33,
    blue=34,
    magenta=35,
    cyan=36,
    white=37,
    crimson=38,
)

def colorize(string, color, bold=True, highlight=False):
    """
    Colorize a string.

    This function was originally written by John Schulman.
    """
    attr = []
    num = color2num[color]
    if highlight:
        num += 10
    attr.append(str(num))
    if bold:
        attr.append("1")
    return "\x1b[%sm%s\x1b[0m" % (";".join(attr), string)

def visual_hex_diff(bstr_1, bstr_2, hex_names=("HEX 1", "HEX 2")):
    SEP = "   |   "
    print("  ", f"{hex_names[0]}".ljust(16 + 7), hex_names[1], sep=SEP)
    # block level
    hex_1, hex_2 = textwrap.wrap(bstr_1.hex(), 16), textwrap.wrap(bstr_2.hex(), 16)
    for i, (block_1, block_2) in enumerate(zip(hex_1, hex_2)):
        # byte level
        block_1, block_2 = textwrap.wrap(block_1, 2), textwrap.wrap(block_2, 2)
        block_2 = [colorize(v2, "red" if v1 != v2 else "green") for v1, v2 in zip(block_1, block_2)]
        print(str(i).rjust(2), " ".join(block_1).ljust(16 + 7), " ".join(block_2).ljust( 16 + 7), sep=SEP)

# Create random string

In [2]:
import random

random.seed(0)

PLAINTEXT_BYTES = 64
BLOCK_SIZE = 128 // 8 # AES block size

plaintext = bytes(random.randrange(256) for _ in range(PLAINTEXT_BYTES))

# save plaintext
with open("h1_plaintext_origin", "wb") as f:
    f.write(plaintext)

print(plaintext.hex(" "))

c5 d7 14 84 f8 cf 9b f4 b7 6f 47 90 47 30 80 4b 9e 32 25 a9 f1 33 b5 de a1 68 f4 e2 85 1f 07 2f cc 00 fc aa 7c a6 20 61 71 7a 48 e5 2e 29 a3 fa 37 9a 95 3f aa 68 93 e3 2e c5 a2 7b 94 5e 60 5f


## Available ciphers in openssl enc

In [3]:
!openssl enc --list | grep -aes-128

-aes-128-cbc               -aes-128-cfb               -aes-128-cfb1             
-aes-128-cfb8              -aes-128-ctr               -aes-128-ecb              
-aes-128-ofb               -aes-192-cbc               -aes-192-cfb              


In [4]:
BLOCK_SIZE = 128

# generate random key and iv
KEY = bytes(random.randrange(256) for _ in range(BLOCK_SIZE // 8))
IV = bytes(random.randrange(256) for _ in range(BLOCK_SIZE // 8))
print(KEY.hex(), IV.hex())

1085f3232d424c1329c88d786ed68ce6 fcb62aa63bf9ab617c088a3b70be57aa


In [5]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.padding import PKCS7

def encrypt_corrupt_then_decrypt(plaintext, mode=modes.ECB(), corrupt_fn=lambda x: x):
    # create cipher
    cipher = Cipher(algorithms.AES(KEY), mode)
    encryptor, decryptor = cipher.encryptor(), cipher.decryptor()
    padder, unpadder = PKCS7(128).padder(), PKCS7(128).unpadder()

    # encrypt
    plaintext = padder.update(plaintext) + padder.finalize() if len(plaintext) % 16 else plaintext
    ciphertext = encryptor.update(plaintext) + encryptor.finalize()

    # corrupt
    ciphertext = corrupt_fn(ciphertext)

    # decrypt
    deciphertext = decryptor.update(ciphertext) + decryptor.finalize()
    deciphertext = unpadder.update(deciphertext) + unpadder.finalize() if len(plaintext) % 16 else deciphertext
    return ciphertext, deciphertext

In [6]:
def visual_bits_diff(a, b):
    bit_str = lambda x: list(bin(int(x,16))[2:].rjust(8, "0"))
    a, b = bit_str(a), bit_str(b)
    b = [colorize(v2, "red" if v1 != v2 else "green") for v1, v2 in zip(a, b)]
    print("Before Flipping: \t{}\nAfter Flipping: \t{}".format(" ".join(a), " ".join(b)))

def flip_one_bit(x, index=0, bit_position=0):
    int_list = list(x)
    original_int = int_list[index]
    int_list[index] ^= (1 << bit_position)
    visual_bits_diff(bytes([original_int]).hex(), bytes([int_list[index]]).hex())
    return bytes(int_list)

In [7]:
flip_one_bit(b"01", 0, 3)

Before Flipping: 	0 0 1 1 0 0 0 0
After Flipping: 	0 0 1 1 1 0 0 0


b'81'

In [8]:
flip_one_bit(b"0", 0, 7)

Before Flipping: 	0 0 1 1 0 0 0 0
After Flipping: 	1 0 1 1 0 0 0 0


b'\xb0'

## ECB

In [9]:
from functools import partial

ciphertext, deciphertext = encrypt_corrupt_then_decrypt(plaintext, modes.ECB())
corrupted_ciphertext, corrupted_deciphertext = encrypt_corrupt_then_decrypt(plaintext, modes.ECB(), partial(flip_one_bit, index=27, bit_position=0))
visual_hex_diff(ciphertext, corrupted_ciphertext, ["ciphertext", "corrupted ciphertext"])
visual_hex_diff(deciphertext, corrupted_deciphertext, ["deciphertext", "corrupted deciphertext"])

Before Flipping: 	0 1 0 1 1 1 0 0
After Flipping: 	0 1 0 1 1 1 0 1
     |   ciphertext                |   corrupted ciphertext
 0   |   b0 7a 37 e2 9c 41 78 5f   |   b0 7a 37 e2 9c 41 78 5f
 1   |   81 94 05 f3 b4 60 d7 9f   |   81 94 05 f3 b4 60 d7 9f
 2   |   7a de 5d 3d 61 1d c7 7f   |   7a de 5d 3d 61 1d c7 7f
 3   |   45 63 d6 5c b1 83 98 83   |   45 63 d6 5d b1 83 98 83
 4   |   0e 42 8c 06 9e da 56 e6   |   0e 42 8c 06 9e da 56 e6
 5   |   32 67 4d b9 c2 ac 54 bd   |   32 67 4d b9 c2 ac 54 bd
 6   |   8e db 04 cd 53 4a 44 7c   |   8e db 04 cd 53 4a 44 7c
 7   |   90 11 b3 8c 68 8f db c5   |   90 11 b3 8c 68 8f db c5
     |   deciphertext              |   corrupted deciphertext
 0   |   c5 d7 14 84 f8 cf 9b f4   |   c5 d7 14 84 f8 cf 9b f4
 1   |   b7 6f 47 90 47 30 80 4b   |   b7 6f 47 90 47 30 80 4b
 2   |   9e 32 25 a9 f1 33 b5 de   |   71 2f b7 62 85 97 a5 be
 3   |   a1 68 f4 e2 85 1f 07 2f   |   3b 6a 63 68 2e 65 04 ac
 4   |   cc 00 fc aa 7c a6 20 61   |   cc 00 fc aa 7c a

## CBC

In [10]:
ciphertext, deciphertext = encrypt_corrupt_then_decrypt(plaintext, modes.CBC(IV))
corrupted_ciphertext, corrupted_deciphertext = encrypt_corrupt_then_decrypt(plaintext, modes.CBC(IV), partial(flip_one_bit, index=27, bit_position=1))
visual_hex_diff(ciphertext, corrupted_ciphertext, ["ciphertext", "corrupted ciphertext"])
visual_hex_diff(deciphertext, corrupted_deciphertext, ["deciphertext", "corrupted deciphertext"])

Before Flipping: 	0 0 0 1 1 0 0 1
After Flipping: 	0 0 0 1 1 0 1 1
     |   ciphertext                |   corrupted ciphertext
 0   |   2c f6 2e 7e 50 95 57 6b   |   2c f6 2e 7e 50 95 57 6b
 1   |   44 60 f1 3c 25 fb 75 c2   |   44 60 f1 3c 25 fb 75 c2
 2   |   7d 1e 2b cb bd 6f f8 d2   |   7d 1e 2b cb bd 6f f8 d2
 3   |   97 79 65 19 0c f2 5c 80   |   97 79 65 1b 0c f2 5c 80
 4   |   c5 ed db 5a 21 d9 d2 da   |   c5 ed db 5a 21 d9 d2 da
 5   |   12 2c 57 e0 96 26 41 13   |   12 2c 57 e0 96 26 41 13
 6   |   ba 93 95 b5 9a 3e e9 23   |   ba 93 95 b5 9a 3e e9 23
 7   |   b1 b7 d1 37 9c 0f ca cc   |   b1 b7 d1 37 9c 0f ca cc
     |   deciphertext              |   corrupted deciphertext
 0   |   c5 d7 14 84 f8 cf 9b f4   |   c5 d7 14 84 f8 cf 9b f4
 1   |   b7 6f 47 90 47 30 80 4b   |   b7 6f 47 90 47 30 80 4b
 2   |   9e 32 25 a9 f1 33 b5 de   |   e6 1e 39 bd 68 ce 92 3e
 3   |   a1 68 f4 e2 85 1f 07 2f   |   45 c5 f0 80 94 6d 66 74
 4   |   cc 00 fc aa 7c a6 20 61   |   cc 00 fc aa 7c a

In [13]:
visual_bits_diff("e5","e7")

Before Flipping: 	1 1 1 0 0 1 0 1
After Flipping: 	1 1 1 0 0 1 1 1


Observation: 
1) Complete corruption of the corresponding block of plaintext 
2) Inverts the corresponding bit in the following block of plaintext
3) The rest of the blocks remains intact

## OFB

In [12]:
ciphertext, deciphertext = encrypt_corrupt_then_decrypt(plaintext, modes.OFB(IV))
corrupted_ciphertext, corrupted_deciphertext = encrypt_corrupt_then_decrypt(plaintext, modes.OFB(IV), partial(flip_one_bit, index=27, bit_position=2))
visual_hex_diff(ciphertext, corrupted_ciphertext, ["ciphertext", "corrupted ciphertext"])
visual_hex_diff(deciphertext, corrupted_deciphertext, ["deciphertext", "corrupted deciphertext"])

Before Flipping: 	0 0 0 1 1 0 0 1
After Flipping: 	0 0 0 1 1 1 0 1
     |   ciphertext                |   corrupted ciphertext
 0   |   c4 06 4c 9d 13 e4 22 66   |   c4 06 4c 9d 13 e4 22 66
 1   |   28 b0 83 9d c2 0b f5 b7   |   28 b0 83 9d c2 0b f5 b7
 2   |   39 87 84 10 dd fa f6 ea   |   39 87 84 10 dd fa f6 ea
 3   |   42 3c 8f 19 a6 be 37 3a   |   42 3c 8f 1d a6 be 37 3a
 4   |   dc aa 6c 32 bf 23 e4 72   |   dc aa 6c 32 bf 23 e4 72
 5   |   3a e0 9a e8 19 49 c5 4d   |   3a e0 9a e8 19 49 c5 4d
 6   |   cd 94 a9 d8 62 65 19 ed   |   cd 94 a9 d8 62 65 19 ed
 7   |   56 8a 34 7a f3 6a 36 8f   |   56 8a 34 7a f3 6a 36 8f
     |   deciphertext              |   corrupted deciphertext
 0   |   c5 d7 14 84 f8 cf 9b f4   |   c5 d7 14 84 f8 cf 9b f4
 1   |   b7 6f 47 90 47 30 80 4b   |   b7 6f 47 90 47 30 80 4b
 2   |   9e 32 25 a9 f1 33 b5 de   |   9e 32 25 a9 f1 33 b5 de
 3   |   a1 68 f4 e2 85 1f 07 2f   |   a1 68 f4 e6 85 1f 07 2f
 4   |   cc 00 fc aa 7c a6 20 61   |   cc 00 fc aa 7c a

In [14]:
visual_bits_diff("e2","e6")

Before Flipping: 	1 1 1 0 0 0 1 0
After Flipping: 	1 1 1 0 0 1 1 0


### CFB

In [15]:
ciphertext, deciphertext = encrypt_corrupt_then_decrypt(plaintext, modes.CFB(IV))
corrupted_ciphertext, corrupted_deciphertext = encrypt_corrupt_then_decrypt(plaintext, modes.CFB(IV), partial(flip_one_bit, index=27, bit_position=3))
visual_hex_diff(ciphertext, corrupted_ciphertext, ["ciphertext", "corrupted ciphertext"])
visual_hex_diff(deciphertext, corrupted_deciphertext, ["deciphertext", "corrupted deciphertext"])

Before Flipping: 	0 1 0 0 1 0 1 1
After Flipping: 	0 1 0 0 0 0 1 1
     |   ciphertext                |   corrupted ciphertext
 0   |   c4 06 4c 9d 13 e4 22 66   |   c4 06 4c 9d 13 e4 22 66
 1   |   28 b0 83 9d c2 0b f5 b7   |   28 b0 83 9d c2 0b f5 b7
 2   |   94 b9 81 8b cd f5 9b 6b   |   94 b9 81 8b cd f5 9b 6b
 3   |   40 ac c5 4b cd 8d ab 9b   |   40 ac c5 43 cd 8d ab 9b
 4   |   56 a5 15 06 e2 ba 6b 72   |   56 a5 15 06 e2 ba 6b 72
 5   |   b6 76 aa 29 76 3c 12 4d   |   b6 76 aa 29 76 3c 12 4d
 6   |   aa 11 52 f1 e9 48 5c 63   |   aa 11 52 f1 e9 48 5c 63
 7   |   4f 56 b8 52 24 05 65 ad   |   4f 56 b8 52 24 05 65 ad
     |   deciphertext              |   corrupted deciphertext
 0   |   c5 d7 14 84 f8 cf 9b f4   |   c5 d7 14 84 f8 cf 9b f4
 1   |   b7 6f 47 90 47 30 80 4b   |   b7 6f 47 90 47 30 80 4b
 2   |   9e 32 25 a9 f1 33 b5 de   |   9e 32 25 a9 f1 33 b5 de
 3   |   a1 68 f4 e2 85 1f 07 2f   |   a1 68 f4 ea 85 1f 07 2f
 4   |   cc 00 fc aa 7c a6 20 61   |   42 26 36 a9 d9 0

In [16]:
visual_bits_diff("e2","ea")

Before Flipping: 	1 1 1 0 0 0 1 0
After Flipping: 	1 1 1 0 1 0 1 0


Observation:
1) Complete corruption of the next block of plaintext 
2) Inverts the corresponding bit in the block of plaintext
3) The rest of the blocks remains intact

### CTR

In [17]:
ciphertext, deciphertext = encrypt_corrupt_then_decrypt(plaintext, modes.CTR(IV))
corrupted_ciphertext, corrupted_deciphertext = encrypt_corrupt_then_decrypt(plaintext, modes.CTR(IV), partial(flip_one_bit, index=27, bit_position=6))
visual_hex_diff(ciphertext, corrupted_ciphertext, ["ciphertext", "corrupted ciphertext"])
visual_hex_diff(deciphertext, corrupted_deciphertext, ["deciphertext", "corrupted deciphertext"])

Before Flipping: 	1 0 0 0 0 0 0 0
After Flipping: 	1 1 0 0 0 0 0 0
     |   ciphertext                |   corrupted ciphertext
 0   |   c4 06 4c 9d 13 e4 22 66   |   c4 06 4c 9d 13 e4 22 66
 1   |   28 b0 83 9d c2 0b f5 b7   |   28 b0 83 9d c2 0b f5 b7
 2   |   64 db 65 95 15 55 98 a5   |   64 db 65 95 15 55 98 a5
 3   |   9e 0d 5f 80 fe 3b 1b e4   |   9e 0d 5f c0 fe 3b 1b e4
 4   |   8b 14 62 bb d4 d0 ca 89   |   8b 14 62 bb d4 d0 ca 89
 5   |   45 7d 5c 02 e1 83 91 cf   |   45 7d 5c 02 e1 83 91 cf
 6   |   6b f6 eb e5 fe 22 36 00   |   6b f6 eb e5 fe 22 36 00
 7   |   e8 1f 53 10 95 6f 50 36   |   e8 1f 53 10 95 6f 50 36
     |   deciphertext              |   corrupted deciphertext
 0   |   c5 d7 14 84 f8 cf 9b f4   |   c5 d7 14 84 f8 cf 9b f4
 1   |   b7 6f 47 90 47 30 80 4b   |   b7 6f 47 90 47 30 80 4b
 2   |   9e 32 25 a9 f1 33 b5 de   |   9e 32 25 a9 f1 33 b5 de
 3   |   a1 68 f4 e2 85 1f 07 2f   |   a1 68 f4 a2 85 1f 07 2f
 4   |   cc 00 fc aa 7c a6 20 61   |   cc 00 fc aa 7c a

In [18]:
visual_bits_diff("e2","a2")

Before Flipping: 	1 1 1 0 0 0 1 0
After Flipping: 	1 0 1 0 0 0 1 0
